In [4]:
!unzip ./raw_data.zip -d ./

Archive:  ./raw_data.zip
  inflating: ./raw_test.csv          
  inflating: ./raw_train.csv         


In [6]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 21.2 MB/s            
     |████████████████████████████████| 67 kB 7.2 MB/s             
     |████████████████████████████████| 6.5 MB 32.3 MB/s            
     |████████████████████████████████| 895 kB 66.4 MB/s            


In [39]:
!pip install datasets

     |████████████████████████████████| 325 kB 25.4 MB/s            
     |████████████████████████████████| 133 kB 88.1 MB/s            
     |████████████████████████████████| 211 kB 89.1 MB/s            
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.4.0
    Uninstalling fsspec-2021.4.0:
      Successfully uninstalled fsspec-2021.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2021.4.0 requires fsspec==2021.04.0, but you have fsspec 2022.1.0 which is incompatible.


In [8]:
import pandas as pd

In [17]:
train = pd.read_csv('raw_train.csv', index_col=0)
test = pd.read_csv('raw_test.csv', index_col=0)

In [18]:
train.drop(columns=['subject_id'], inplace=True)
train.rename(columns={'features': 'text', 'target': 'label'}, inplace=True)

In [23]:
test.drop(columns=['subject_id'], inplace=True)
test.rename(columns={'features': 'text', 'target': 'label'}, inplace=True)

In [64]:
test.dropna(inplace=True)
train.dropna(inplace=True)

In [71]:
train.isnull().sum()

text     0
label    0
dtype: int64

In [78]:
from transformers import BertTokenizer, AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [66]:
def preprocess_function(row):
    return tokenizer(row["text"], truncation=True)

In [67]:
from datasets import Dataset

In [68]:
train_data = Dataset.from_pandas(train, preserve_index=False)
test_data = Dataset.from_pandas(test, preserve_index=False)

In [80]:
train_dataset = train_data.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [81]:
test_dataset = test_data.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [86]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [83]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [84]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [87]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [ ]:
trainer.train()